## 55_二叉树的深度

题目一：

输入一个二叉树的根节点，求该树的深度。从根节点到叶节点依次经过的节点（含根，叶节点）形成树的一条路径，最长路径的长度为树的深度。
<img src="images/img55.png" style="width: 200px;"/>


### 分析
Naive的方法是求出从顶端到每个leaf的所有路径，就能得到最长的路径。在34题“二叉树中和为某一值的路径”中，我们详细讨论了如何记录树中的路径。这种思路的代码量比较大。

从递归的角度来理解树的深度。对只有一个节点的树，它的深度为1。如果根节点只有left没有right，那么它的深度为左子树的深度+1。（只有右子树同理）。如果左右都有，那么该树的深度是左右子树的深度的较大值+1。

[//]: # (<img src="images/img123.png" style="width: 500px;"/>)

In [2]:
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None


def tree_depth(root: TreeNode):
    if root is None:
        return 0
    
    n_left = tree_depth(root.left)
    n_right = tree_depth(root.right)
    
    return n_left + 1 if n_left > n_right else n_right + 1

In [3]:
root1 = TreeNode(1)
root1.left = TreeNode(2)
root1.right = TreeNode(3)

root1.left.left = TreeNode(4)
root1.left.right = TreeNode(5)
root1.left.right.right = TreeNode(7)

root1.right.right = TreeNode(6)

print(tree_depth(root1))

4


### 题目二：平衡二叉树

输入一棵二叉树的根节点，判断该树是不是平衡二叉树。如果某二叉树中任意节点的左，右子树的深度相差不超过1，那么它就是一棵平衡二叉树。例如题目一中的图6.2就是平衡二叉树。

### 分析：
Naive的方法：有了题目一的基础之后，我们很容易想到，在遍历树的每个节点的时候，调用函数tree_depth得到左右子树的深度。如果任意节点的左右子树的深度差不超过1，那么根据定义它就是一棵平衡二叉树。但是这种方法需要重复遍历节点多次

In [6]:
def is_balanced(root):
    if root is None:
        return True
    
    left = tree_depth(root.left)
    right = tree_depth(root.right)
    diff = left - right
    if diff > 1 or diff < -1:
        return False
    
    return is_balanced(root.left) and is_balanced(root.right)

print(is_balanced(root1))

True


### 每个节点只遍历一次的解法
如果用**后序遍历**的方式遍历二叉树的每个节点，那么在遍历到一个节点之前我们就已经遍历了它的左右子树。只要在遍历每个节点的时候记录它的深度（某一节点的深度=它到leaf的路径的长度），我们就可以一边遍历，一边判断每个节点是不是平衡的。

观察是如何在一个简单的后序遍历基础上发展成下面的算法的：
```python
seq = []
def post_order_traverse(root: TreeNode):
    if root.left is None and root.right is None:
        seq.append(root.val)
        return

    # go all the way left
    if root.left is not None:
        post_order_traverse(root.left)

    # go right
    if root.right is not None:
        post_order_traverse(root.right)

    if root is not None:
        seq.append(root.val)
```

In [9]:
class Solution:
    def __init__(self):
        self.seq = []

    def post_order_traverse(self, root: TreeNode):
        if root.left is None and root.right is None:
            self.seq.append(root.val)
            depth = 1
            return True, depth

        # go left all the way
        if root.left is not None:
            (left_ok, left) = self.post_order_traverse(root.left)
        else:
            left = 0
            left_ok = True

        # go right
        if root.right is not None:
            (right_ok, right) = self.post_order_traverse(root.right)
        else:
            right = 0
            right_ok = True

        if root is not None:
            self.seq.append(root.val)
            if left_ok and right_ok:
                diff = abs(left - right)
                if diff <= 1:
                    depth = 1 + max(left, right)
                    return True, depth
        return False, 'unknown'

### Test
          1                      1
        /   \                   / \  
       2     3                 2   3
      / \     \               / \   \
     4   5     6             4   5   6
        / \                     / \
       7   8                   7   8
                                    \
                                     9

In [10]:
root1 = TreeNode(1)
root1.left = TreeNode(2)
root1.right = TreeNode(3)

root1.left.left = TreeNode(4)
root1.left.right = TreeNode(5)

root1.left.right.left = TreeNode(7)
root1.left.right.right = TreeNode(8)

root1.right.right = TreeNode(6)

sol = Solution()
result = sol.post_order_traverse(root1)
print(sol.seq)
print("is_balanced={}, max_depth={}".format(result[0], result[1]))

# ==============================
root2 = TreeNode(1)
root2.left = TreeNode(2)
root2.right = TreeNode(3)

root2.left.left = TreeNode(4)
root2.left.right = TreeNode(5)

root2.left.right.left = TreeNode(7)
root2.left.right.right = TreeNode(8)
root2.left.right.right.right = TreeNode(9)

root2.right.right = TreeNode(6)

sol = Solution()
result = sol.post_order_traverse(root2)
print(sol.seq)
print("is_balanced={}, max_depth={}".format(result[0], result[1]))

[4, 7, 8, 5, 2, 6, 3, 1]
is_balanced=True, max_depth=4
[4, 7, 9, 8, 5, 2, 6, 3, 1]
is_balanced=False, max_depth=unknown
